# Importación de librerias

In [1]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Carga de datos y previsualización

In [3]:
ruta = '/content/drive/MyDrive/clientes_sucios_grande.csv'
datos = pd.read_csv(ruta)
print(datos.head())
print(datos.info())

  Nombre  Edad     Ciudad  Ingresos Estado Civil
0   Juan  19.0  Barcelona       NaN       Casado
1  Elena   NaN     Madrid    1244.0        Viudo
2  Marta  50.0        NaN       NaN   Divorciada
3  Marta   NaN     Madrid    7216.0   Divorciada
4  Laura  35.0     Bilbao    1837.0   Divorciada
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Nombre        100 non-null    object 
 1   Edad          70 non-null     float64
 2   Ciudad        86 non-null     object 
 3   Ingresos      32 non-null     float64
 4   Estado Civil  88 non-null     object 
dtypes: float64(2), object(3)
memory usage: 4.0+ KB
None


# Limpieza

In [4]:
datos = datos.drop_duplicates()
datos["Edad"].fillna(datos["Edad"].mean(), inplace=True)
datos["Ingresos"].fillna(datos["Ingresos"].mean(), inplace=True)
datos["Estado Civil"].fillna("Desconocido", inplace=True)

/tmp/ipython-input-606215050.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  datos["Edad"].fillna(datos["Edad"].mean(), inplace=True)
/tmp/ipython-input-606215050.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', 

# Enriquecimiento

In [5]:
# Creamos una nueva columna llamada "Nivel_Ingreso"
datos["Nivel_Ingreso"] = pd.cut(
    datos["Ingresos"],
    bins=[0,2000,4000,6000],
    labels=["Bajo","Medio","Alto"]
)

# Preprocesamiento

In [6]:
# Normalizamos ingresos entre 0 y 1.
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
datos[["Ingresos"]] = scaler.fit_transform(datos[["Ingresos"]])
# Codificamos ciudad (One-Hot Encoding)
datos = pd.get_dummies(datos, columns=["Ciudad"])


# Guardado de datos

In [7]:
datos.to_csv("clientes_limpios.csv", index = False)

# Comparación de datos

In [8]:
# 1. Carga de archivos
sucio = pd.read_csv("/content/drive/MyDrive/clientes_sucios_grande.csv")
limpio = pd.read_csv('clientes_limpios.csv')

# 2. Mostrar estructura general
print("🔹 Columnas en archivo sucio:", sucio.columns.tolist())
print("🔹 Columnas en archivo limpio:", limpio.columns.tolist())
print("\n🔸 Tipos de datos (sucio):\n", sucio.dtypes)
print("\n🔸 Tipos de datos (limpio):\n", limpio.dtypes)

# 3. Comparar número de filas y columnas
print(f"\n📊 Dimensiones - Sucio: {sucio.shape}, Limpio: {limpio.shape}")

# 4. Comparar valores nulos
print("\n🧼 Valores nulos por columna (sucio):\n", sucio.isnull().sum())
print("\n✅ Valores nulos por columna (limpio):\n", limpio.isnull().sum())

# 5. Estadísticas descriptivas
print("\n📈 Estadísticas descriptivas (sucio):\n", sucio.describe(include='all'))
print("\n📈 Estadísticas descriptivas (limpio):\n", limpio.describe(include='all'))

# 6. Ver filas duplicadas en el dataset sucio
duplicados = sucio[sucio.duplicated()]
print(f"\n📎 Filas duplicadas en sucio: {len(duplicados)}")

# 7. Filas que están en sucio y ya no están en limpio (posiblemente eliminadas)
diferencia = pd.concat([sucio, limpio]).drop_duplicates(keep=False)
print(f"\n🧐 Filas diferentes entre ambos archivos: {len(diferencia)}")
print(diferencia.head())

# 8. Comparar directamente fila por fila (si tienen mismo orden e índices)
if sucio.shape == limpio.shape:
    diferencias_elemento = (sucio != limpio).sum()
    print("\n🔍 Diferencias por columna (elemento por elemento):\n", diferencias_elemento)
else:
    print("\n❗Los archivos tienen diferente número de filas/columnas. Comparación directa no es posible.")

# 9. Comparar columnas faltantes o añadidas
columnas_sucio = set(sucio.columns)
columnas_limpio = set(limpio.columns)
print("\n➖ Columnas que están en sucio y no en limpio:", columnas_sucio - columnas_limpio)
print("➕ Columnas que están en limpio y no en sucio:", columnas_limpio - columnas_sucio)


🔹 Columnas en archivo sucio: ['Nombre', 'Edad', 'Ciudad', 'Ingresos', 'Estado Civil']
🔹 Columnas en archivo limpio: ['Nombre', 'Edad', 'Ingresos', 'Estado Civil', 'Nivel_Ingreso', 'Ciudad_Barcelona', 'Ciudad_Bilbao', 'Ciudad_Granada', 'Ciudad_Madrid', 'Ciudad_Málaga', 'Ciudad_Sevilla', 'Ciudad_Sevilla ', 'Ciudad_Valencia', 'Ciudad_Zaragoza', 'Ciudad_madrid']

🔸 Tipos de datos (sucio):
 Nombre           object
Edad            float64
Ciudad           object
Ingresos        float64
Estado Civil     object
dtype: object

🔸 Tipos de datos (limpio):
 Nombre               object
Edad                float64
Ingresos            float64
Estado Civil         object
Nivel_Ingreso        object
Ciudad_Barcelona       bool
Ciudad_Bilbao          bool
Ciudad_Granada         bool
Ciudad_Madrid          bool
Ciudad_Málaga          bool
Ciudad_Sevilla         bool
Ciudad_Sevilla         bool
Ciudad_Valencia        bool
Ciudad_Zaragoza        bool
Ciudad_madrid          bool
dtype: object

📊 Dimensiones

In [9]:
diferencia.to_csv("diferencias_detectadas.csv", index=False)
